# 4.1 Tensorboard 可视化好帮手 1

注意: 本节内容会用到浏览器, 而且与 tensorboard 兼容的浏览器是 Google Chrome. 使用其他的浏览器不保证所有内容都能正常显示.
学会用 Tensorflow 自带的 tensorboard 去可视化我们所建造出来的神经网络是一个很好的学习理解方式. 用最直观的流程图告诉你你的神经网络是长怎样,有助于你发现编程中间的问题和疑问.

In [68]:
from __future__ import print_function
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

#### 定义函数

In [69]:
def add_layer(inputs, in_size, out_size, activation_function=None):
    # add one more layer and return the output of this layer
    with tf.name_scope('layer'):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='W')
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b')
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b, )
        return outputs

首先从 Input 开始：
这里指定的名称将来会在可视化的图层inputs中显示出来
使用with tf.name_scope('inputs')可以将xs和ys包含进来，形成一个大的图层，图层的名字就是with tf.name_scope()方法里的参数。

In [70]:
with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32, [None, 1], name='x_input')
    ys = tf.placeholder(tf.float32, [None, 1], name='y_input')

#### add hidden layer

In [71]:
l1 = add_layer(xs, 1, 10, activation_function=tf.nn.relu)

#### add output layer

In [72]:
prediction = add_layer(l1, 10, 1, activation_function=None)

最后编辑loss部分：将with tf.name_scope()添加在loss上方，并为它起名为loss

In [73]:
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction),
                                        reduction_indices=[1]))

使用with tf.name_scope()再次对train_step部分进行编辑,如下：

In [74]:
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

sess = tf.Session()

我们需要使用 tf.summary.FileWriter() (tf.train.SummaryWriter() 这种方式已经在 tf >= 0.12 版本中摒弃) 将上面‘绘画’出的图保存到一个目录中，以方便后期在浏览器中可以浏览。 这个方法中的第二个参数需要使用sess.graph ， 因此我们需要把这句话放在获取session的后面。 这里的graph是将前面定义的框架信息收集起来，然后放在logs/目录下面。

In [75]:
if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:  # tensorflow version < 0.12
    writer = tf.train.SummaryWriter('logs/', sess.graph)
else: # tensorflow version >= 0.12
    writer = tf.summary.FileWriter("logs/", sess.graph)

#### tf.initialize_all_variables() no long valid from
#### 2017-03-02 if using tensorflow >= 0.12

In [76]:
if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
    init = tf.initialize_all_variables()
else:
    init = tf.global_variables_initializer()
sess.run(init)

direct to the local dir and run this in terminal:
$ tensorboard --logdir=D:\0ilraypan\git\tutorials\tensorflowTUT\logs